# Bahdanau Attention
- Seq2Seq의 컨텍스트 벡터가 고정된 길이로 정보를 압축하는 것이 손실을 야기한다고 주장, 짧은 문장은 괜찮을지 모르나 문장이 길어질수록 성능이 저하된다!
- 그래서 그는 Encoder의 최종 state값만을 사용하는 방식이 아닌, **매 스텝의 hidden state를 활용해 context vector를 구축**하는 attention 매커니즘을 제안
- **decoder가 현재 시점 i에서 보기에 인코더의 어느 부분 j가 중요한가?**
- 이 가중치가 바로 attention이다!


$Score_alignment = W_combine * tanh(W_decoder * H_decoder + W_encoder * H_encoder)$

In [3]:
import tensorflow as tf

In [4]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W_decoder = tf.keras.layers.Dense(units)
        self.W_encoder = tf.keras.layers.Dense(units)
        self.W_combine = tf.keras.layers.Dense(1)
        
    def call(self, H_encoder, H_decoder):
        print('[ H_encoder ] shape : ', H_encoder.shape)
        
        H_encoder = self.W_encoder(H_encoder)  # Encoder의 모든 스텝에 대한 Hidden State를 100차원의 벡터 공간으로 매핑 (1, 10, 100)
        print('[ W_encoder X H_encoder ] Shape:',H_encoder.shape)
        
        print("\n[ H_decoder ] Shape:", H_decoder.shape)
        H_decoder = tf.expand_dims(H_decoder, 1)
        H_decoder = self.W_decoder(H_decoder)  # Decoder의 현재 스텝에 대한 Hidden State 역시 100차원의 벡터 공간으로 매핑 (1, 1, 100)
        print("[ W_decoder X H_decoder ] Shape:", H_decoder.shape)
        
        score = self.W_combine(tf.nn.tanh(H_decoder + H_encoder))  #  두 State의 합으로 정의된 Score (1, 10, 1) 를 구함
        print('[ Score_alignment ] Shape:', score.shape)
        
        attention_weights = tf.nn.softmax(score, axis=1)  # Softmax를 거쳐 나온 값은 0-1 사이의 값으로 각 단어가 차지하는 비중을 말함
        print('\n최종 Weight: \n', attention_weights.numpy())
        
        context_vector = attention_weights * H_decoder
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        return context_vector, attention_weights

In [5]:
W_size = 100

print(f'Hidden State를 {W_size}차원으로 Mapping\n')

attention = BahdanauAttention(W_size)

enc_state = tf.random.uniform((1, 10, 512))
dec_state = tf.random.uniform((1, 512))

_ = attention(enc_state, dec_state)

Hidden State를 100차원으로 Mapping

[ H_encoder ] shape :  (1, 10, 512)
[ W_encoder X H_encoder ] Shape: (1, 10, 100)

[ H_decoder ] Shape: (1, 512)
[ W_decoder X H_decoder ] Shape: (1, 1, 100)
[ Score_alignment ] Shape: (1, 10, 1)

최종 Weight: 
 [[[0.14594649]
  [0.15569305]
  [0.07879995]
  [0.08378106]
  [0.10317663]
  [0.06685784]
  [0.11261033]
  [0.06850183]
  [0.1024792 ]
  [0.08215365]]]


위 결과(최종 weight)는 랜덤값을 사용했기 때문에 비중이 비슷비슷하지만 실제 단어로 적용시켜보면 **유사한 단어에 높은 비중을 할당**하게 된다.

<br><br><br><br>

# Luong Attention

In [6]:
class LuongAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(LuongAttention, self).__init__()
        self.W_combine = tf.keras.layers.Dense(units)

    def call(self, H_encoder, H_decoder):
        print("[ H_encoder ] Shape:", H_encoder.shape)

        WH = self.W_combine(H_encoder)
        print("[ W_encoder X H_encoder ] Shape:", WH.shape)

        H_decoder = tf.expand_dims(H_decoder, 1)
        alignment = tf.matmul(WH, tf.transpose(H_decoder, [0, 2, 1]))
        print("[ Score_alignment ] Shape:", alignment.shape)

        attention_weights = tf.nn.softmax(alignment, axis=1)
        print("\n최종 Weight:\n", attention_weights.numpy())

        attention_weights = tf.squeeze(attention_weights, axis=-1)
        context_vector = tf.matmul(attention_weights, H_encoder)

        return context_vector, attention_weights

In [7]:
emb_dim = 512

attention = LuongAttention(emb_dim)

enc_state = tf.random.uniform((1, 10, emb_dim))
dec_state = tf.random.uniform((1, emb_dim))

_ = attention(enc_state, dec_state)

[ H_encoder ] Shape: (1, 10, 512)


ResourceExhaustedError: OOM when allocating tensor with shape[512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]